In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
X_data=pd.read_csv("A:\\Coding\\ML\\Kaggle\\House-price-prediction-master\\train.csv",index_col='Id')
X_test=pd.read_csv("A:\\Coding\\ML\\Kaggle\\House-price-prediction-master\\test.csv",index_col='Id')
# SalePrice missing rows removal
X_data.dropna(axis=0,subset=['SalePrice'],inplace=True)

y=X_data.SalePrice
X_data.drop(['SalePrice'],axis=1,inplace=True)

X_train,X_valid,y_train,y_valid=train_test_split(X_data,y,train_size=0.8,test_size=0.2,random_state=1)

In [2]:
# numerical columns
numerical_cols=[col for col in X_train.columns if X_train[col].dtype in ['int64','float64']]
# object columns
object_cols=[col for col in X_train.columns if X_train[col].dtype== 'object']
good_label_cols=[col for col in X_train.columns if set(X_valid[col]).issubset(set(X_train[col])) and X_train[col].dtype=='object']
#good_label_cols=[col for col in good_label_cols if X_train[col].nunique() <10]
print(len(good_label_cols))
bad_cols=list(set(object_cols)-set(good_label_cols))


40


In [3]:
# numerical cols pre processing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
pd.options.mode.chained_assignment = None 
my_imputer=SimpleImputer(strategy='mean')
X_train_numeric=X_train[numerical_cols]
X_valid_numeric=X_valid[numerical_cols]
X_train_numeric=pd.DataFrame(my_imputer.fit_transform(X_train_numeric),columns=X_train_numeric.columns)
X_valid_numeric=pd.DataFrame(my_imputer.transform(X_valid_numeric),columns=X_train_numeric.columns)

# my_scaler=StandardScaler()
my_scaler=MinMaxScaler()


X_train_numeric=pd.DataFrame(my_scaler.fit_transform(X_train_numeric),columns=X_train_numeric.columns)
X_valid_numeric=pd.DataFrame(my_scaler.transform(X_valid_numeric),columns=X_train_numeric.columns)

In [4]:
# object columns pre processing 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
X_train_cat=X_train[good_label_cols]
X_valid_cat=X_valid[good_label_cols]
imputer2=SimpleImputer(strategy='most_frequent')
X_train_cat=pd.DataFrame(imputer2.fit_transform(X_train_cat),columns=X_train_cat.columns)
X_valid_cat=pd.DataFrame(imputer2.transform(X_valid_cat),columns=X_train_cat.columns)
Encoder=OneHotEncoder(handle_unknown='ignore',sparse=False)
X_train_encode=pd.DataFrame(Encoder.fit_transform(X_train_cat))
X_valid_encode=pd.DataFrame(Encoder.transform(X_valid_cat))
X_train_encode.index=X_train_cat.index
X_valid_encode.index=X_valid_cat.index
# Encoder=OrdinalEncoder()
# X_train_encode=pd.DataFrame(Encoder.fit_transform(X_train_cat))
# X_valid_encode=pd.DataFrame(Encoder.transform(X_valid_cat))
# X_train_encode.columns=X_train_cat.columns
# X_valid_encode.columns=X_valid_cat.columns

X_train_final=pd.concat([X_train_numeric,X_train_encode],axis=1)
X_valid_final=pd.concat([X_valid_numeric,X_valid_encode],axis=1)

In [5]:
#model fit and predict 
#model fit and predict 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

model=RandomForestRegressor(n_estimators=180,random_state=0)
model.fit(X_train_final,y_train)
pred=model.predict(X_valid_final)
print(mean_absolute_error(y_valid,pred))

16299.366210045662


In [11]:
#model fit and predict 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

model=XGBRegressor(n_estimators=450,random_state=0,learning_rate=.04)
model.fit(X_train_final,y_train)
pred=model.predict(X_valid_final)
print(mean_absolute_error(y_valid,pred))

14999.695406142979


In [12]:

X_test_numeric=X_test[numerical_cols]

X_test_cat=X_test[good_label_cols]

X_test_numeric=pd.DataFrame(my_imputer.transform(X_test_numeric),columns=X_test_numeric.columns)

X_test_numeric=pd.DataFrame(my_scaler.transform(X_test_numeric),columns=X_test_numeric.columns)

X_test_cat=pd.DataFrame(imputer2.transform(X_test_cat))

X_test_encode=pd.DataFrame(Encoder.transform(X_test_cat))


X_test_encode.index=X_test_cat.index

X_test_final=pd.concat([X_test_numeric,X_test_encode],axis=1)
preds=model.predict(X_test_final)

In [13]:
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds})
output.to_csv('xgboost_variable_submission.csv', index=False)